In [ ]:
!pip install transformers # if not already installed

In [52]:
import os
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import torch
import transformers
import nltk
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from nltk.corpus import stopwords
import joblib


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Data Loading
path = '/content/drive/My Drive/Intensity_data/'

# Load the CSV files into pandas DataFrames
angriness_df = pd.read_csv(os.path.join(path, 'angriness.csv'))
happiness_df = pd.read_csv(os.path.join(path, 'happiness.csv'))
sadness_df = pd.read_csv(os.path.join(path, 'sadness.csv'))

# Display the data to check the content
print(angriness_df.head())
print('-'*75)
print(happiness_df.head())
print('-'*75)
print(sadness_df.head())

                                             content  intensity
0  Sometimes I’m not angry, I’m hurt and there’s ...  angriness
1                     Not available for busy people☺  angriness
2  I do not exist to impress the world. I exist t...  angriness
3  Everything is getting expensive except some pe...  angriness
4       My phone screen is brighter than my future 🙁  angriness
---------------------------------------------------------------------------
                                             content  intensity
0  Wants to know how the hell I can remember word...  happiness
1  Love is a long sweet dream & marriage is an al...  happiness
2  The world could be amazing when you are slight...  happiness
3  My secret talent is getting tired without doin...  happiness
4  Khatarnaak Whatsapp Status Ever… Can\’t talk, ...  happiness
---------------------------------------------------------------------------
                                             content intensity
0  Never hurt peo

In [5]:
print(angriness_df.shape)
print(angriness_df.columns)

(696, 2)
Index(['content', 'intensity'], dtype='object')


In [6]:
print(happiness_df.shape)
print(happiness_df.columns)

(708, 2)
Index(['content', 'intensity'], dtype='object')


In [7]:
print(sadness_df.shape)
print(sadness_df.columns)

(635, 2)
Index(['content', 'intensity'], dtype='object')


In [8]:
# Add a label column for each DataFrame
angriness_df['label'] = 0  # Label for angriness
happiness_df['label'] = 1  # Label for happiness
sadness_df['label'] = 2    # Label for sadness

print(angriness_df.shape)
print(happiness_df.shape)
print(sadness_df.shape)

(696, 3)
(708, 3)
(635, 3)


In [9]:
# Combine the DataFrames
df = pd.concat([angriness_df, happiness_df, sadness_df])

# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

# Display the combined DataFrame
print(df.head())

                                             content  intensity  label
0  Don't be much closer to anybody because a smal...    sadness      2
1          Anger is only one letter short of Danger.  angriness      0
2  If I tell you "I love you" then I actually do ...  happiness      1
3  Success seems to be connected with action. Suc...  happiness      1
4  I hate that I cry when I’m frustrated because ...  angriness      0


In [10]:
# finding missing values if any
df.isna().sum()
# df.shape

content      0
intensity    0
label        0
dtype: int64

In [23]:

def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)    # Remove HTML tags
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    return text

df['content'] = df['content'].apply(clean_text)

In [24]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

df['content'] = df['content'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [28]:
X = df['content']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [29]:
# Vectorize text data using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [30]:
# Train the SVM model
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train_tfidf, y_train)


SVC(kernel='linear', random_state=42)

In [31]:
# Predict on the test set
y_pred = svm_model.predict(X_test_tfidf)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.74      0.80      0.77       139
           1       0.75      0.75      0.75       142
           2       0.81      0.73      0.77       127

    accuracy                           0.76       408
   macro avg       0.76      0.76      0.76       408
weighted avg       0.76      0.76      0.76       408



In [57]:
# Save the model and vectorizer
joblib.dump(svm_model, 'svm_intensity_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

model_save_path = '/content/drive/My Drive/svm_intensity_model.pkl'

# Save the XGBoost model
joblib.dump(svm_model, model_save_path)


['/content/drive/My Drive/svm_intensity_model.pkl']

In [58]:
test_sentences = [
    "I am very happy with the service!",
    "This is the worst experience I've ever had.",
    "I feel so sad and disappointed with the outcome.",
    "The product quality is amazing and I'm extremely satisfied!",
    "I'm angry because my order was not delivered on time.",
    "I am overjoyed with the support I received!",
    "This is terrible, I am very upset.",
    "I couldn't be happier with my purchase.",
    "I'm feeling very down and nothing seems to help.",
    "The experience was okay, nothing special."
]

In [59]:
# Load the model and vectorizer for deployment
# svm_model = joblib.load('svm_intensity_model.pkl')
# vectorizer = joblib.load('tfidf_vectorizer.pkl')

def predict_intensity_rf(text):
    text = clean_text(text)
    text = remove_stopwords(text)
    text_tfidf = vectorizer.transform([text])
    prediction = svm_model.predict(text_tfidf)
    return prediction[0]

# Predict intensities for example sentences with Random Forest model
for sentence in test_sentences:
    intensity = predict_intensity_rf(sentence)
    print(f"Sentence: {sentence}\nPredicted Intensity: {intensity}\n")


Sentence: I am very happy with the service!
Predicted Intensity: 1

Sentence: This is the worst experience I've ever had.
Predicted Intensity: 0

Sentence: I feel so sad and disappointed with the outcome.
Predicted Intensity: 2

Sentence: The product quality is amazing and I'm extremely satisfied!
Predicted Intensity: 1

Sentence: I'm angry because my order was not delivered on time.
Predicted Intensity: 0

Sentence: I am overjoyed with the support I received!
Predicted Intensity: 1

Sentence: This is terrible, I am very upset.
Predicted Intensity: 1

Sentence: I couldn't be happier with my purchase.
Predicted Intensity: 0

Sentence: I'm feeling very down and nothing seems to help.
Predicted Intensity: 0

Sentence: The experience was okay, nothing special.
Predicted Intensity: 0



**Training with Naive_Bayes model**

In [35]:
# Train the Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)


MultinomialNB()

In [36]:
# Predict on the test set
y_pred = nb_model.predict(X_test_tfidf)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.72      0.78      0.74       139
           1       0.79      0.52      0.63       142
           2       0.64      0.82      0.72       127

    accuracy                           0.70       408
   macro avg       0.71      0.71      0.70       408
weighted avg       0.72      0.70      0.70       408



In [49]:
# Save the model and vectorizer
joblib.dump(nb_model, 'nb_intensity_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
model_save_path = '/content/drive/My Drive/nb_intensity_model.pkl'

# Save the XGBoost model
joblib.dump(nb_model, model_save_path)

['/content/drive/My Drive/nb_intensity_model.pkl']

In [60]:



for sentence in test_sentences:
    intensity = predict_intensity(sentence)
    print(f"Sentence: {sentence}\nPredicted Intensity: {intensity}\n")

Sentence: I am very happy with the service!
Predicted Intensity: 1

Sentence: This is the worst experience I've ever had.
Predicted Intensity: 2

Sentence: I feel so sad and disappointed with the outcome.
Predicted Intensity: 2

Sentence: The product quality is amazing and I'm extremely satisfied!
Predicted Intensity: 1

Sentence: I'm angry because my order was not delivered on time.
Predicted Intensity: 0

Sentence: I am overjoyed with the support I received!
Predicted Intensity: 1

Sentence: This is terrible, I am very upset.
Predicted Intensity: 2

Sentence: I couldn't be happier with my purchase.
Predicted Intensity: 1

Sentence: I'm feeling very down and nothing seems to help.
Predicted Intensity: 0

Sentence: The experience was okay, nothing special.
Predicted Intensity: 2



**Random Forest Classifier**

In [53]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)

RandomForestClassifier(random_state=42)

In [54]:
# Predict on the test set
y_pred_rf = rf_model.predict(X_test_tfidf)

# Evaluate the model
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.75      0.81      0.78       139
           1       0.74      0.78      0.76       142
           2       0.82      0.70      0.76       127

    accuracy                           0.76       408
   macro avg       0.77      0.76      0.76       408
weighted avg       0.77      0.76      0.76       408



In [55]:
model_save_path = '/content/drive/My Drive/rf_intensity_model.pkl'

# Save the XGBoost model
joblib.dump(rf_model, model_save_path)

['/content/drive/My Drive/rf_intensity_model.pkl']

In [56]:
# Load the Random Forest model and vectorizer for deployment
# rf_model = joblib.load('rf_intensity_model.pkl')
# vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Function to predict intensity with Random Forest model
def predict_intensity_rf(text):
    text = clean_text(text)
    text = remove_stopwords(text)
    text_tfidf = vectorizer.transform([text])
    prediction = rf_model.predict(text_tfidf)
    return prediction[0]

# Predict intensities for example sentences with Random Forest model
for sentence in test_sentences:
    intensity = predict_intensity_rf(sentence)
    print(f"Sentence: {sentence}\nPredicted Intensity: {intensity}\n")


Sentence: I am very happy with the service!
Predicted Intensity: 1

Sentence: This is the worst experience I've ever had.
Predicted Intensity: 0

Sentence: I feel so sad and disappointed with the outcome.
Predicted Intensity: 2

Sentence: The product quality is amazing and I'm extremely satisfied!
Predicted Intensity: 1

Sentence: I'm angry because my order was not delivered on time.
Predicted Intensity: 0

Sentence: I am overjoyed with the support I received!
Predicted Intensity: 1

Sentence: This is terrible, I am very upset.
Predicted Intensity: 1

Sentence: I couldn't be happier with my purchase.
Predicted Intensity: 1

Sentence: I'm feeling very down and nothing seems to help.
Predicted Intensity: 0

Sentence: The experience was okay, nothing special.
Predicted Intensity: 2



**XGBoost model training**

In [44]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_model.fit(X_train_tfidf, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [45]:
# Predict on the test set
y_pred_xgb = xgb_model.predict(X_test_tfidf)

# Evaluate the model
print(classification_report(y_test, y_pred_xgb))


              precision    recall  f1-score   support

           0       0.77      0.82      0.79       139
           1       0.73      0.79      0.76       142
           2       0.86      0.72      0.78       127

    accuracy                           0.78       408
   macro avg       0.79      0.78      0.78       408
weighted avg       0.78      0.78      0.78       408



In [48]:
# Save the XGBoost model
# Path to save the model in Google Drive
model_save_path = '/content/drive/My Drive/xgb_intensity_model.pkl'

# Save the XGBoost model
joblib.dump(xgb_model, model_save_path)



['/content/drive/My Drive/xgb_intensity_model.pkl']

In [47]:


# Function to predict intensity with XGBoost model
def predict_intensity_xgb(text):
    text = clean_text(text)
    text = remove_stopwords(text)
    text_tfidf = vectorizer.transform([text])
    prediction = xgb_model.predict(text_tfidf)
    return prediction[0]

# Predict intensities for example sentences with XGBoost model
for sentence in test_sentences:
    intensity = predict_intensity_xgb(sentence)
    print(f"Sentence: {sentence}\nPredicted Intensity: {intensity}\n")


Sentence: I am very happy with the service!
Predicted Intensity: 1

Sentence: This is the worst experience I've ever had.
Predicted Intensity: 1

Sentence: I feel so sad and disappointed with the outcome.
Predicted Intensity: 2

Sentence: The product quality is amazing and I'm extremely satisfied!
Predicted Intensity: 0

Sentence: I'm angry because my order was not delivered on time.
Predicted Intensity: 0

Sentence: I am overjoyed with the support I received!
Predicted Intensity: 1

Sentence: This is terrible, I am very upset.
Predicted Intensity: 1

Sentence: I couldn't be happier with my purchase.
Predicted Intensity: 1

Sentence: I'm feeling very down and nothing seems to help.
Predicted Intensity: 0

Sentence: The experience was okay, nothing special.
Predicted Intensity: 2



**Model performance comparison**

In [64]:
# SVM
svm_report = classification_report(y_test, svm_model.predict(X_test_tfidf), target_names=['Angriness', 'Happiness', 'Sadness'], output_dict=True)
svm_precisions = [svm_report[label]['precision'] for label in ['Angriness', 'Happiness', 'Sadness']]
svm_recalls = [svm_report[label]['recall'] for label in ['Angriness', 'Happiness', 'Sadness']]
svm_f1_scores = [svm_report[label]['f1-score'] for label in ['Angriness', 'Happiness', 'Sadness']]

# Random Forest
rf_report = classification_report(y_test, rf_model.predict(X_test_tfidf), target_names=['Angriness', 'Happiness', 'Sadness'], output_dict=True)
rf_precisions = [rf_report[label]['precision'] for label in ['Angriness', 'Happiness', 'Sadness']]
rf_recalls = [rf_report[label]['recall'] for label in ['Angriness', 'Happiness', 'Sadness']]
rf_f1_scores = [rf_report[label]['f1-score'] for label in ['Angriness', 'Happiness', 'Sadness']]

# XGBoost
xgb_report = classification_report(y_test, xgb_model.predict(X_test_tfidf), target_names=['Angriness', 'Happiness', 'Sadness'], output_dict=True)
xgb_precisions = [xgb_report[label]['precision'] for label in ['Angriness', 'Happiness', 'Sadness']]
xgb_recalls = [xgb_report[label]['recall'] for label in ['Angriness', 'Happiness', 'Sadness']]
xgb_f1_scores = [xgb_report[label]['f1-score'] for label in ['Angriness', 'Happiness', 'Sadness']]

# Naive Bayes
nb_report = classification_report(y_test, nb_model.predict(X_test_tfidf), target_names=['Angriness', 'Happiness', 'Sadness'], output_dict=True)
nb_precisions = [nb_report[label]['precision'] for label in ['Angriness', 'Happiness', 'Sadness']]
nb_recalls = [nb_report[label]['recall'] for label in ['Angriness', 'Happiness', 'Sadness']]
nb_f1_scores = [nb_report[label]['f1-score'] for label in ['Angriness', 'Happiness', 'Sadness']]

# Print the comparison table
print("|   Model              | Precision (Angriness)      | Precision (Happiness)     | Precision (Sadness)       | Recall (Angriness)     | Recall (Happiness)      | Recall (Sadness)       | F1-score (Angriness)     | F1-score (Happiness)    | F1-score (Sadness) |")
print("|----------------------|----------------------------|---------------------------|---------------------------|------------------------|-------------------------|------------------------|--------------------------|-------------------------|--------------------|")
print(f"| **SVM**             | {svm_precisions[0]:.4f}    | {svm_precisions[1]:.4f}   | {svm_precisions[2]:.4f}   | {svm_recalls[0]:.4f}   | {svm_recalls[1]:.4f}    | {svm_recalls[2]:.4f}   | {svm_f1_scores[0]:.4f}   | {svm_f1_scores[1]:.4f}  | {svm_f1_scores[2]:.4f}|")
print(f"| **Random Forest**   | {rf_precisions[0]:.4f}     | {rf_precisions[1]:.4f}    | {rf_precisions[2]:.4f}    | {rf_recalls[0]:.4f}    | {rf_recalls[1]:.4f}     | {rf_recalls[2]:.4f}    | {rf_f1_scores[0]:.4f}    | {rf_f1_scores[1]:.4f}   | {rf_f1_scores[2]:.4f} |")
print(f"| **XGBoost**         | {xgb_precisions[0]:.4f}    | {xgb_precisions[1]:.4f}   | {xgb_precisions[2]:.4f}   | {xgb_recalls[0]:.4f}   | {xgb_recalls[1]:.4f}    | {xgb_recalls[2]:.4f}   | {xgb_f1_scores[0]:.4f}   | {xgb_f1_scores[1]:.4f}  | {xgb_f1_scores[2]:.4f}|")
print(f"| **Naive Bayes**     | {nb_precisions[0]:.4f}     | {nb_precisions[1]:.4f}    | {nb_precisions[2]:.4f}    | {nb_recalls[0]:.4f}    | {nb_recalls[1]:.4f}     | {nb_recalls[2]:.4f}    | {nb_f1_scores[0]:.4f}    | {nb_f1_scores[1]:.4f}   | {nb_f1_scores[2]:.4f} |")


|   Model              | Precision (Angriness)      | Precision (Happiness)     | Precision (Sadness)       | Recall (Angriness)     | Recall (Happiness)      | Recall (Sadness)       | F1-score (Angriness)     | F1-score (Happiness)    | F1-score (Sadness) |
|----------------------|----------------------------|---------------------------|---------------------------|------------------------|-------------------------|------------------------|--------------------------|-------------------------|--------------------|
| **SVM**             | 0.7351    | 0.7465   | 0.8087   | 0.7986   | 0.7465    | 0.7323   | 0.7655   | 0.7465  | 0.7686|
| **Random Forest**   | 0.7467     | 0.7400    | 0.8241    | 0.8058    | 0.7817     | 0.7008    | 0.7751    | 0.7603   | 0.7574 |
| **XGBoost**         | 0.7651    | 0.7320   | 0.8585   | 0.8201   | 0.7887    | 0.7165   | 0.7917   | 0.7593  | 0.7811|
| **Naive Bayes**     | 0.7152     | 0.7872    | 0.6380    | 0.7770    | 0.5211     | 0.8189    | 0.7448    

| Model             | Precision (Angriness) | Precision (Happiness) | Precision (Sadness) | Recall (Angriness) | Recall (Happiness) | Recall (Sadness) | F1-score (Angriness) | F1-score (Happiness) | F1-score (Sadness) |
|-------------------|-----------------------|-----------------------|---------------------|---------------------|---------------------|-------------------|-----------------------|-----------------------|---------------------|
| **SVM**           | 0.7351                | 0.7465                | 0.8087              | 0.7986              | 0.7465              | 0.7323            | 0.7655                | 0.7465                | 0.7686              |
| **Random Forest** | 0.7467                | 0.7400                | 0.8241              | 0.8058              | 0.7817              | 0.7008            | 0.7751                | 0.7603                | 0.7574              |
| **XGBoost**       | 0.7651                | 0.7320                | 0.8585              | 0.8201              | 0.7887              | 0.7165            | 0.7917                | 0.7593                | 0.7811              |
| **Naive Bayes**   | 0.7152                | 0.7872                | 0.6380              | 0.7770              | 0.5211              | 0.8189            | 0.7448                | 0.6271                | 0.7172              |

This table summarizes the performance of each model in terms of precision, recall, and F1-score for predicting the intensity levels of Angriness, Happiness, and Sadness. The higher the values of precision, recall, and F1-score, the better the model's performance for that particular class.

**BERT model finetuning**

In [ ]:
# Tokenization and Encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text
def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, return_tensors="pt")

# Apply the tokenization
tokenized_texts = df['content'].apply(lambda x: tokenize_function(x))

# Convert labels to tensor
labels = torch.tensor(df['label'].values)

# Convert the tokenized texts to tensors
input_ids = torch.cat([x['input_ids'] for x in tokenized_texts])
attention_masks = torch.cat([x['attention_mask'] for x in tokenized_texts])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Split the data into training and testing sets
train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_ids, labels, test_size=0.2, random_state=42)
train_masks, test_masks, _, _ = train_test_split(attention_masks, labels, test_size=0.2, random_state=42)

batch_size = 16

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our test set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [ ]:
print(len(train_dataloader))
print(len(test_dataloader))

102
26


In [ ]:
def train_model_v1(model, train_dataloader, val_dataloader, epochs=10, lr=2e-5, eps=1e-8, patience=3):
    # Initialize the optimizer
    optimizer = AdamW(model.parameters(), lr=lr, eps=eps)

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Create the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    # Loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # Early stopping parameters
    best_val_loss = float('inf')
    early_stopping_counter = 0

    # Training loop
    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        print('-' * 10)

        # Set model to training mode
        model.train()

        total_train_loss = 0
        total_train_accuracy = 0

        for step, batch in enumerate(train_dataloader):
            b_input_ids, b_input_mask, b_labels = tuple(t.cuda() for t in batch)

            model.zero_grad()

            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss
            logits = outputs.logits

            total_train_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()

            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            total_train_accuracy += flat_accuracy(logits, label_ids)

        avg_train_loss = total_train_loss / len(train_dataloader)
        avg_train_accuracy = total_train_accuracy / len(train_dataloader)

        print(f'Training loss: {avg_train_loss}')
        print(f'Training accuracy: {avg_train_accuracy}')

        # Validation
        avg_val_loss, avg_val_accuracy = evaluate_model(model, val_dataloader, loss_fn)

        print(f'Validation loss: {avg_val_loss}')
        print(f'Validation accuracy: {avg_val_accuracy}')

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print('Early stopping triggered')
            break





In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# Save the model
output_dir = '/content/drive/My Drive/Intensity_data'


In [ ]:
def evaluate_model(model, dataloader, loss_fn):
    model.eval()

    total_eval_loss = 0
    total_eval_accuracy = 0

    for batch in dataloader:
        b_input_ids, b_input_mask, b_labels = tuple(t.cuda() for t in batch)

        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss
            logits = outputs.logits

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_loss = total_eval_loss / len(dataloader)
    avg_val_accuracy = total_eval_accuracy / len(dataloader)

    return avg_val_loss, avg_val_accuracy

In [ ]:
def calculate_metrics(model, dataloader):
    model.eval()

    predictions, true_labels = [], []

    for batch in dataloader:
        b_input_ids, b_input_mask, b_labels = tuple(t.cuda() for t in batch)

        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask)
            logits = outputs.logits

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        predictions.append(logits)
        true_labels.append(label_ids)

    flat_predictions = np.concatenate(predictions, axis=0)
    flat_true_labels = np.concatenate(true_labels, axis=0)

    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

    precision, recall, f1, _ = precision_recall_fscore_support(flat_true_labels, flat_predictions, average='weighted')

    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}')

    return precision, recall, f1

In [ ]:
# Initialize BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.to(device)  # Move the model to the GPU

# Train the model
train_model_v1(model, train_dataloader, test_dataloader,epochs=4)

# Evaluate the model
calculate_metrics(model, test_dataloader)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4
----------
Training loss: 0.9117892694239523
Training accuracy: 0.5556372549019608
Validation loss: 0.72658132819029
Validation accuracy: 0.6778846153846154
Epoch 2/4
----------
Training loss: 0.5784151270693424
Training accuracy: 0.7615196078431372
Validation loss: 0.6095993174956396
Validation accuracy: 0.7548076923076923
Epoch 3/4
----------
Training loss: 0.3679913754994963
Training accuracy: 0.8700980392156863
Validation loss: 0.6192839180047696
Validation accuracy: 0.7668269230769231
Epoch 4/4
----------
Training loss: 0.25442876586434887
Training accuracy: 0.9147875816993465
Validation loss: 0.6300898767434634
Validation accuracy: 0.7740384615384616
Precision: 0.776541996997166
Recall: 0.7720588235294118
F1-Score: 0.7716760925007389


(0.776541996997166, 0.7720588235294118, 0.7716760925007389)

In [ ]:
# Train the model
import time
t_0 = time.time()
train_model_v1(model, train_dataloader, test_dataloader,epochs=20)
t_final = time.time()
print(f"Time taken to train the model is {t_final - t_0} ")
# Evaluate the model
calculate_metrics(model, test_dataloader)
model.save_pretrained(output_dir)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20
----------
Training loss: 0.26370121944038305
Training accuracy: 0.9141748366013073
Validation loss: 0.706290908730947
Validation accuracy: 0.7764423076923077
Epoch 2/20
----------
Training loss: 0.1673115490862698
Training accuracy: 0.9466911764705882
Validation loss: 0.8464498892426491
Validation accuracy: 0.7764423076923077
Epoch 3/20
----------
Training loss: 0.08713690307922661
Training accuracy: 0.9767156862745098
Validation loss: 1.03806850199516
Validation accuracy: 0.7956730769230769
Epoch 4/20
----------
Training loss: 0.07745685910933889
Training accuracy: 0.9791666666666666
Validation loss: 1.1384737491607666
Validation accuracy: 0.7788461538461539
Early stopping triggered
Time taken to train the model is 619.7203950881958 
Precision: 0.780861018009993
Recall: 0.7769607843137255
F1-Score: 0.7773429190503146


In [ ]:
def train_model_v2(model, train_dataloader, val_dataloader, epochs=10, lr=2e-5, eps=1e-8, patience=3, dropout=0.1):
    # Initialize the optimizer
    optimizer = AdamW(model.parameters(), lr=lr, eps=eps)

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Create the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    # Loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # Early stopping parameters
    best_val_loss = float('inf')
    early_stopping_counter = 0

    # Training loop
    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        print('-' * 10)

        # Set model to training mode
        model.train()

        total_train_loss = 0
        total_train_accuracy = 0

        for step, batch in enumerate(train_dataloader):
            b_input_ids, b_input_mask, b_labels = tuple(t.cuda() for t in batch)

            model.zero_grad()

            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss
            logits = outputs.logits

            total_train_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()

            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            total_train_accuracy += flat_accuracy(logits, label_ids)

        avg_train_loss = total_train_loss / len(train_dataloader)
        avg_train_accuracy = total_train_accuracy / len(train_dataloader)

        print(f'Training loss: {avg_train_loss}')
        print(f'Training accuracy: {avg_train_accuracy}')

        # Validation
        avg_val_loss, avg_val_accuracy = evaluate_model(model, val_dataloader, loss_fn)

        print(f'Validation loss: {avg_val_loss}')
        print(f'Validation accuracy: {avg_val_accuracy}')

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print('Early stopping triggered')
            break

In [ ]:
# Train the model
import time
new_dropout = 0.2
t_0 = time.time()
train_model_v2(model, train_dataloader, test_dataloader,epochs=20, dropout = new_dropout )
t_final = time.time()
print(f"Time taken to train the model is {t_final - t_0} ")
# Evaluate the model
calculate_metrics(model, test_dataloader)
model.save_pretrained(output_dir)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20
----------
Training loss: 0.02256160699114701
Training accuracy: 0.9938725490196079
Validation loss: 1.707851602194401
Validation accuracy: 0.7836538461538461
Epoch 2/20
----------
Training loss: 0.02818279397901029
Training accuracy: 0.9908088235294118
Validation loss: 1.6453570906932538
Validation accuracy: 0.7668269230769231
Epoch 3/20
----------
Training loss: 0.040472110452052824
Training accuracy: 0.9889705882352942
Validation loss: 1.618588863657071
Validation accuracy: 0.7764423076923077
Epoch 4/20
----------
Training loss: 0.019756603777518738
Training accuracy: 0.991421568627451
Validation loss: 1.718175578575868
Validation accuracy: 0.7860576923076923
Epoch 5/20
----------
Training loss: 0.013124158037766499
Training accuracy: 0.9932598039215687
Validation loss: 1.7241086782171176
Validation accuracy: 0.7908653846153846
Epoch 6/20
----------
Training loss: 0.03092801536959265
Training accuracy: 0.9895833333333334
Validation loss: 1.6869039237499237
Validation accu

In [ ]:
# Train the model
import time
new_dropout = 0.4
t_0 = time.time()
train_model_v2(model, train_dataloader, test_dataloader,epochs=20, dropout = new_dropout )
t_final = time.time()
print(f"Time taken to train the model is {t_final - t_0} ")
# Evaluate the model
calculate_metrics(model, test_dataloader)
model_03 = model
model_03.save_pretrained(output_dir)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20
----------
Training loss: 0.03211386226700829
Training accuracy: 0.9889705882352942
Validation loss: 1.7771748396066518
Validation accuracy: 0.7764423076923077
Epoch 2/20
----------
Training loss: 0.015536144327366035
Training accuracy: 0.9926470588235294
Validation loss: 1.8720603534557785
Validation accuracy: 0.7740384615384616
Epoch 3/20
----------
Training loss: 0.021275737923668415
Training accuracy: 0.9895833333333334
Validation loss: 1.9861431213525624
Validation accuracy: 0.7620192307692307
Epoch 4/20
----------
Training loss: 0.023598316161844518
Training accuracy: 0.9895833333333334
Validation loss: 1.960076249562777
Validation accuracy: 0.7740384615384616
Early stopping triggered
Time taken to train the model is 619.4590418338776 
Precision: 0.7740450198992126
Recall: 0.7720588235294118
F1-Score: 0.7723256213019846


In [ ]:
model_03 = model
model_03.save_pretrained(output_dir)

Since the dataset is smaller, BERT finetuning seems to be overfitting even with significant amount of regularization. Therefore it is safe to surmise that, with smaller datasets, we can choose XGBoost as the best available alternative.